In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [79]:
weather_train = pd.read_csv("weather_train.csv")

In [80]:
weather_train.shape

(139773, 9)

In [81]:
365*24*16-139773

387

全ての場所に対して、366×24時間分の天候データが存在するわけではない。387（365*24*16-139773）時刻分の天候データは存在しない。
学習データを作成するときに、学習期間にない時刻におけるメーターの測定値を予測する時は、その時刻の他のビルにおける天候データが重要になる。要するに天候データがそのビルの時系列情報を含んでいる唯一の情報源なので、
それをnanで用意するということはしたくない。
387時点のデータは何かしたの手法で**水増し**をしたい。

In [82]:
weather_test = pd.read_csv("weather_test.csv")
weather_data = pd.concat([weather_train, weather_test], axis=0).reset_index(drop=True)

In [83]:
weather_test

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6
5,0,2017-01-01 05:00:00,15.6,2.0,12.8,0.0,1022.1,130.0,2.1
6,0,2017-01-01 06:00:00,15.0,0.0,12.8,0.0,1021.7,150.0,1.5
7,0,2017-01-01 07:00:00,15.0,2.0,13.3,0.0,1021.6,0.0,0.0
8,0,2017-01-01 08:00:00,13.3,0.0,12.2,0.0,1021.4,0.0,0.0
9,0,2017-01-01 09:00:00,12.2,4.0,11.7,0.0,1021.4,0.0,0.0


In [34]:
temp_data = pd.DataFrame({"timestamp": pd.date_range('2016-01-01 00:00:00', periods=(366+730)*24, freq='h')}).astype("datetime64")
weather_data["timestamp"] = weather_data["timestamp"].astype("datetime64")
for i in range(4):
    temp_data = pd.concat([temp_data, temp_data],axis=0)
temp_data = temp_data.sort_values("timestamp").reset_index(drop=True)

In [35]:
one_array = np.ones((26304,1))

In [51]:
id_array = one_array * np.arange(16)
id_array_sort = np.sort(id_array.reshape(420864,))
id_array_sort = pd.DataFrame({"site_id": id_array_sort}).astype("int")
temp_data_id = pd.concat([id_array_sort, temp_data], axis=1)

In [52]:
temp_data_id["timestamp"]

0        2016-01-01 00:00:00
1        2016-01-01 00:00:00
2        2016-01-01 00:00:00
3        2016-01-01 00:00:00
4        2016-01-01 00:00:00
                 ...        
420859   2018-12-31 23:00:00
420860   2018-12-31 23:00:00
420861   2018-12-31 23:00:00
420862   2018-12-31 23:00:00
420863   2018-12-31 23:00:00
Name: timestamp, Length: 420864, dtype: datetime64[ns]

In [53]:
weather_data["timestamp"]

0        2016-01-01 00:00:00
1        2016-01-01 01:00:00
2        2016-01-01 02:00:00
3        2016-01-01 03:00:00
4        2016-01-01 04:00:00
                 ...        
417011   2018-12-31 19:00:00
417012   2018-12-31 20:00:00
417013   2018-12-31 21:00:00
417014   2018-12-31 22:00:00
417015   2018-12-31 23:00:00
Name: timestamp, Length: 417016, dtype: datetime64[ns]

In [68]:
weather_augment = pd.merge(temp_data_id, weather_data, on=["timestamp", "site_id"], how="left")

16×(1096)×24(16種類×3年分×24時間)=420864

In [69]:
temp_data_id.shape

(420864, 2)

In [70]:
weather_data.shape

(417016, 9)

In [71]:
weather_augment.shape

(420864, 9)

In [72]:
weather_augment.isnull().sum()

site_id                    0
timestamp                  0
air_temperature         2016
cloud_coverage        222864
dew_temperature         2064
precip_depth_1_hr     148704
sea_level_pressure     32688
wind_direction         20784
wind_speed              2384
dtype: int64

欠損の割合が異なるので埋めるのにしても同一の方法で埋めるのはよくなさそう。

埋め方は、特徴量同士の相関よりも時系列としての意味合いが強いと考えられるので、その特徴量の過去の値を考慮した埋め方にする。時系列モデルにすると手間がかかるので、nanの出現する前の最後の値で欠損を埋めることにする。

In [73]:
weather_augment = weather_augment.fillna(method="ffill")
weather_augment = weather_augment.fillna(method="backfill")

In [74]:
weather_augment.isnull().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [75]:
weather_augment.to_csv("weather_augment.csv")

In [76]:
weather_augment.shape

(420864, 9)

In [77]:
weather_augment.groupby("site_id")["timestamp"].nunique()

site_id
0     1644
1     1644
2     1644
3     1644
4     1644
5     1644
6     1644
7     1644
8     1644
9     1644
10    1644
11    1644
12    1644
13    1644
14    1644
15    1644
Name: timestamp, dtype: int64